<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/BiLSTM-CRF%20Model%20(Char%20%2B%20Word%20Embeddings).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings Bi-LSTM-CRF Model

In [1]:
from google.colab import files
uploaded = files.upload()

Saving prepro.py to prepro (2).py


In [2]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-1vipdykg
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-1vipdykg
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=852e745e5eb1f8e867b6f9728c19a3b7b234521ae2a44a14d950675fa50655b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-72r2aniz/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [3]:
!pip install sklearn_crfsuite

In [0]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring, partial_tags

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras.layers import concatenate


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score

In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [6]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
tags = partial_tags(tags)

setlabel=list(set(tags))
setlabel.remove('O')
        
words= [w.lower() for w in words]

n_words = len(set(words))
print("Number of words in the dataset: ", n_words)
n_tags = len(list(set(tags)))
print("Number of Labels: ", n_tags)

Number of words in the dataset:  24277
Number of Labels:  5


In [7]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(list(set(tags)))}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels LOC (location) is identified by the index: {}".format(tag2idx["ORG"]))

The labels LOC (location) is identified by the index: 2


In [8]:
word2idx = {w: i + 2 for i, w in enumerate(list(set(words)))}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 15555


In [9]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [10]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0].lower()] for w in s] for s in train]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X[0]

array([18283, 15555,  5588, 20313, 22004, 16656,  5122,  8202,  2234,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)

In [11]:
def get_labeler(word):
  if word =="O":
    return word
  elif "MISC" in word:
    return word[-4:]
  else:
    return word[-3:]

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y[0]

array([2, 5, 1, 5, 5, 5, 1, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

## Adding Embeddings

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:

#root_path = "/content/drive/My Drive/glove.6B.50d.txt"
#root_path = "/content/drive/My Drive/glove.6B.100d.txt"
root_path = "/content/drive/My Drive/glove.6B.200d.txt"
#root_path = "/content/drive/My Drive/glove.6B.300d.txt"
#root_path = "/content/drive/My Drive/wiki-news-300d-1M.vec"

embeddings_index={}
f = open(root_path, encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
#create emedding matrix

EMBEDDING=200
embedding_matrix = np.zeros((len(word2idx), EMBEDDING))
for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

## Character Embeddings

In [25]:
chars = set([w_i for w in words for w_i in w])
print ("Number of characters:",len(chars))

char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

max_len_char =10

Number of characters: 59


In [0]:


X_char = []
for sentence in train:
    sent_seq = []
    for i in range(MAX_LEN):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

In [0]:
# Model definition
max_len =124

#inputs for embedding
word_in = Input(shape=(max_len,))
char_in = Input(shape=(max_len, max_len_char,))

#word embedding
emb_word = Embedding(len(word2idx),output_dim=EMBEDDING,
                  weights=[embedding_matrix],input_length=max_len,
                  trainable=False,mask_zero=True)(word_in)

#character embedding
emb_char = TimeDistributed(Embedding(input_dim=len(char2idx), output_dim=20,
                           input_length=max_len_char, mask_zero=True))(char_in)



# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)


#main LSTM 
x = concatenate([emb_word, char_enc])

lstm1 = Bidirectional(LSTM(units=64, return_sequences=True,
                               recurrent_dropout=0.2))(x)

lstm2 = Bidirectional(LSTM(units=64, return_sequences=True,
                               recurrent_dropout=0.2))(lstm1)

model_2 = TimeDistributed(Dense(10, activation="relu"))(lstm2)
crf = CRF(n_tags+1)  # CRF layer
out = crf(model_2)  # output
model_2 = Model([word_in, char_in], out)

In [69]:
model_2.compile(optimizer='adam', loss=crf_loss,metrics=[crf_viterbi_accuracy])
model_2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           (None, 124, 10)      0                                            
__________________________________________________________________________________________________
input_43 (InputLayer)           (None, 124)          0                                            
__________________________________________________________________________________________________
time_distributed_36 (TimeDistri (None, 124, 10, 20)  1220        input_44[0][0]                   
__________________________________________________________________________________________________
embedding_41 (Embedding)        (None, 124, 200)     4855800     input_43[0][0]                   
__________________________________________________________________________________________________
time_distr

In [70]:
%%time
BATCH_SIZE = 200
EPOCHS=16


history = model_2.fit([X,np.array(X_char).reshape((len(X_char), max_len, max_len_char))], 
                      np.array(y), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1)

Train on 11232 samples, validate on 2809 samples
Epoch 1/16
11232/11232 [==============================] - 87s 8ms/step - loss: 33.4895 - crf_viterbi_accuracy: 0.7865 - val_loss: 29.1539 - val_crf_viterbi_accuracy: 0.8660
Epoch 2/16
11232/11232 [==============================] - 79s 7ms/step - loss: 32.9460 - crf_viterbi_accuracy: 0.9000 - val_loss: 28.7999 - val_crf_viterbi_accuracy: 0.9389
Epoch 3/16
11232/11232 [==============================] - 78s 7ms/step - loss: 32.7286 - crf_viterbi_accuracy: 0.9565 - val_loss: 28.7283 - val_crf_viterbi_accuracy: 0.9654
Epoch 4/16
11232/11232 [==============================] - 78s 7ms/step - loss: 32.6828 - crf_viterbi_accuracy: 0.9706 - val_loss: 28.7017 - val_crf_viterbi_accuracy: 0.9709
Epoch 5/16
11232/11232 [==============================] - 78s 7ms/step - loss: 32.6611 - crf_viterbi_accuracy: 0.9756 - val_loss: 28.6920 - val_crf_viterbi_accuracy: 0.9730
Epoch 6/16
11232/11232 [==============================] - 77s 7ms/step - loss: 32.6473

## Predict on Training Set

In [71]:
pred = model_2.predict([X,np.array(X_char).reshape((len(X_char), max_len, max_len_char))], verbose=1)

14041/14041 [==============================] - 199s 14ms/step


In [72]:
# TRain Eval
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted')
print(score)

              precision    recall  f1-score   support

         LOC       0.97      0.96      0.97      8297
        MISC       0.95      0.92      0.93      4593
           O       1.00      1.00      1.00    169578
         ORG       0.95      0.96      0.95     10025
         PAD       1.00      1.00      1.00   1537463
         PER       0.99      0.98      0.98     11128

    accuracy                           1.00   1741084
   macro avg       0.98      0.97      0.97   1741084
weighted avg       1.00      1.00      1.00   1741084

0.9990570835605035


In [73]:
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

        MISC       0.95      0.92      0.93      4593
         ORG       0.95      0.96      0.95     10025
         PER       0.99      0.98      0.98     11128
         LOC       0.97      0.96      0.97      8297

   micro avg       0.97      0.96      0.96     34043
   macro avg       0.96      0.95      0.96     34043
weighted avg       0.97      0.96      0.96     34043

0.9635926281817256


## Predict on Test Set

In [0]:
# Convert each sentence from list of Token to list of word_index
X_test = [[word2idx[w[0].lower()] for w in s] for s in test]
# Padding each sentence to have the same lenght
X_test = pad_sequences(maxlen=MAX_LEN, sequences=X_test, padding="post", value=word2idx["PAD"])
X_test

# Convert Tag/Label to tag_index
y_test = [[tag2idx[get_labeler((w[1]))] for w in s] for s in test]
# Padding each sentence to have the same lenght
y_test = pad_sequences(maxlen=MAX_LEN, sequences=y_test, padding="post", value=tag2idx["PAD"])
# One-Hot encode categorical labels
y_test = [to_categorical(i, num_classes=n_tags+1) for i in y_test]

X_char_test = []
for sentence in test:
    sent_seq = []
    for i in range(MAX_LEN):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char_test.append(np.array(sent_seq))


In [75]:
pred = model_2.predict([X_test,np.array(X_char_test).reshape((len(X_char_test), max_len, max_len_char))], verbose=1)

3453/3453 [==============================] - 49s 14ms/step


In [76]:
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y_test, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)


              precision    recall  f1-score   support

        MISC       0.75      0.78      0.76       918
         ORG       0.87      0.86      0.86      2496
         PER       0.97      0.95      0.96      2773
         LOC       0.89      0.91      0.90      1925

   micro avg       0.89      0.90      0.89      8112
   macro avg       0.87      0.88      0.87      8112
weighted avg       0.89      0.90      0.89      8112

0.8942745200556365
